In [ ]:
import sys
import os
from matplotlib.colors import from_levels_and_colors
sys.path.append(os.environ['GOTMWORK_ROOT']+'/tools', )
from gotmanalysis import *
np.seterr(all='raise')
%matplotlib inline

In [ ]:
casename = 'JRA55-do_Global_dampV5d'
forcing_reg_type = 'BG12'
tmname = 'KPP-CVMix'
update_data = False
plot_figure = True

In [ ]:
# check forcing_reg_type
fr_list = ['BG12', 'LF17']
if forcing_reg_type not in fr_list:
    print('Forcing regime {} not supported. Stop.'.format(forcing_reg_type))

In [ ]:
# lists
timetag_list = ['20090101-20090131',
                '20090201-20090228',
                '20090301-20090331',
                '20090401-20090430',
                '20090501-20090531',
                '20080601-20080630',
                '20080701-20080731',
                '20080801-20080831',
                '20080901-20080930',
                '20081001-20081031',
                '20081101-20081130',
                '20081201-20081231']
ntag = len(timetag_list)

In [ ]:
# paths
fig_root = os.environ['GOTMFIG_ROOT']+'/'+casename


In [ ]:
# get data
mon_lat = []
mon_lon = []
mon_time = []
mon_laturb = []
mon_bflux = []
mon_ustar = []
mon_hbl = []
for j in np.arange(ntag):
    timetag = timetag_list[j]
    # read surface forcing  data
    s1data_root = os.environ['GOTMRUN_ROOT']+'/'+casename+'/VR1m_DT600s_'+timetag
    s2data_root = os.environ['GOTMFIG_ROOT']+'/data/'+casename+'/VR1m_DT600s_'+timetag
    os.makedirs(s2data_root, exist_ok=True)
    os.makedirs(fig_root, exist_ok=True)
    basepath = s1data_root+'/'+tmname
    s2data_name = s2data_root+'/data_mapts_surface_forcing_'+tmname+'.npz'
    if update_data or not os.path.isfile(s2data_name):
        # update data
        print('Updating data for {} ...'.format(timetag))
        loclist = sorted(os.listdir(basepath))
        pathlist = [basepath+'/'+x+'/gotm_out_s1.nc' for x in loclist]
        godmobj = GOTMOutputDataMap(pathlist)
        laturb = np.zeros([godmobj.ncase, godmobj.ntime-1])
        bflux = np.zeros([godmobj.ncase, godmobj.ntime-1])
        ustar = np.zeros([godmobj.ncase, godmobj.ntime-1])
        hbl = np.zeros([godmobj.ncase, godmobj.ntime-1])
        lat = godmobj.lat
        lon = godmobj.lon
        time = godmobj.time[1:]
        for i in np.arange(godmobj.ncase):
            if np.mod(i, 100) == 0:
                print('{:6.2f} %'.format(i/godmobj.ncase*100.0))
            tmp = GOTMOutputData(godmobj._paths[i], init_time_location=False)
            laturb[i,:] = tmp.read_timeseries('La_Turb', ignore_time=True).data[1:]
            bflux[i,:] = tmp.read_timeseries('bflux', ignore_time=True).data[1:]
            ustar[i,:] = tmp.read_timeseries('u_taus', ignore_time=True).data[1:]
            hbl[i,:] = tmp.read_timeseries('mld_deltaR', ignore_time=True).data[1:]
        # save data
        np.savez(s2data_name, laturb=laturb, bflux=bflux, ustar=ustar, hbl=hbl,
                 lon=lon, lat=lat, time=time)
    else:
        # read data
        tmp = np.load(s2data_name)
        lat = tmp['lat']
        lon = tmp['lon']
        time = tmp['time']
        laturb = tmp['laturb']
        bflux = tmp['bflux']
        ustar = tmp['ustar']
        hbl = tmp['hbl']
    # append to monthly lists
    mon_lat.append(lat)
    mon_lon.append(lon)
    mon_time.append(time)
    mon_laturb.append(laturb)
    mon_bflux.append(bflux)
    mon_ustar.append(ustar)
    mon_hbl.append(hbl)
    

In [ ]:
def plot_dist_4p(hst, xi, yi, axis=None, filled=False, **kwargs):
    """Plot bi-dimensional histogram. Show the contours of the
       histogram which enclose the highest 30%, 60%, 90% and 99%
       centered distribution.
       
    :his: (2D numpy array) bi-dimensional histogram
    :xi: (1D numpy array) centers of x dimension
    :yi: (1D numpy array) centers of y dimension
    :axis: (matplotlib.axes, optional) axis to plot figure on
    :filled: (bool) filled contour if True
    :return: (matplotlib figure object) figure
    
    """
    # use curret axis if not specified
    if axis is None:
        axis = plt.gca()
    hsum = np.sum(hst)
    hlist = -np.sort(-hst.flatten())/hsum
    hcum = np.cumsum(hlist)
    vl = [0.3, 0.6, 0.9, 0.99]
    nv = len(vl)
    vlev = np.zeros(nv)
    for i in np.arange(nv):
        ind = np.argmin(abs(hcum-vl[i]))
        vlev[i] = hlist[ind]
    pdfData = hst/hsum
    pdfData[pdfData==0] = 1e-12
    if not filled:
        fig = axis.contour(xi, yi, np.log10(np.transpose(pdfData)), levels=np.log10(vlev[::-1]), **kwargs)
    else:
        fig = axis.contourf(xi, yi, np.log10(np.transpose(pdfData)), levels=np.log10(vlev[::-1]), **kwargs)
    return fig

In [ ]:
# Figure 1: regime diagram following Belcher et al., 2012

# turbulent Langmuir number
fl_laturb = np.concatenate([mon_laturb[i].flatten() for i in np.arange(ntag)])
# surface buoyancy flux
fl_bflux = np.concatenate([mon_bflux[i].flatten() for i in np.arange(ntag)])
# friction velocity
fl_ustar = np.concatenate([mon_ustar[i].flatten() for i in np.arange(ntag)])
# boundary layer depth
fl_hbl = np.concatenate([mon_hbl[i].flatten() for i in np.arange(ntag)])
# remove data points where friction velocity is zero
inds = fl_ustar==0
print('Invalid data points: {:6.2f}%'.format(np.sum(inds)/fl_ustar.size*100))
fl_laturb[inds] = np.nan
fl_bflux[inds] = np.nan
fl_ustar[inds] = np.nan
fl_hbl[inds] = np.nan
fl_laturb = fl_laturb[~np.isnan(fl_laturb)]
fl_bflux = fl_bflux[~np.isnan(fl_bflux)]
fl_ustar = fl_ustar[~np.isnan(fl_ustar)]
fl_hbl = fl_hbl[~np.isnan(fl_hbl)]
# h_b/L_L
fl_hLL = -fl_bflux*fl_hbl/(fl_ustar**3)*fl_laturb**2

# 
xdata = fl_laturb
ydata = fl_hLL
# remove data points where surface buoyancy flux is positive (stable, h_b/L_L<0) 
inds = ydata < 0
nneg = np.sum(inds) 
print('Negative h_b/L_L: {:6.2f}%'.format(nneg/ydata.size*100))
xdata[inds] = np.nan
ydata[inds] = np.nan
xdata = xdata[~np.isnan(xdata)]
ydata = ydata[~np.isnan(ydata)]

# get the bi-dimensional histogram in log-log space
xdata = np.log10(xdata)
ydata = np.log10(ydata)
# range of power
xpr = [-1, 1]
ypr = [-3, 3]
# range
xlims = [10**i for i in xpr]
ylims = [10**i for i in ypr]
hist, xi, yi, c = plt.hist2d(xdata, ydata, range=(xpr,ypr), bins=100)
# clean the figure
plt.clf()
# get the centers from the edges
xi = 0.5*(xi[0:-1]+xi[1:])
yi = 0.5*(yi[0:-1]+yi[1:])
# convert back to actual values
xi = 10**xi
yi = 10**yi

# plot figure
fig = plt.figure()
fig.set_size_inches(4.5, 4)

# background following Fig. 3 of Belcher et al., 2012
nx = 500
ny = 500
xx = np.logspace(xpr[0], xpr[1], nx)
yy = np.logspace(ypr[0], ypr[1], ny)
zz1 = np.zeros([nx, ny])
zz2 = np.zeros([nx, ny])
zz3 = np.zeros([nx, ny])
for i in np.arange(nx):
    for j in np.arange(ny):
        zz1[i,j] = 2*(1-np.exp(-0.5*xx[i]))
        zz2[i,j] = 0.22*xx[i]**(-2)
        zz3[i,j] = 0.3*xx[i]**(-2)*yy[j]
zz = zz1 + zz2 + zz3
plt.contourf(xx, yy, np.transpose(np.log10(zz)),
             levels=[-0.1, 0, 0.1, 0.25, 0.5, 1, 2, 3],
             cmap='summer', extend='both')
plt.contour(xx, yy, np.transpose(np.log10(zz)),
             levels=[-0.1, 0, 0.1, 0.25, 0.5, 1, 2, 3],
             colors='darkgray')
plt.contour(xx, yy, np.transpose(zz1/zz), levels=0.9, colors='k',
           linestyles='-', linewidths=2)
plt.contour(xx, yy, np.transpose(zz2/zz), levels=0.9, colors='k',
           linestyles='-', linewidths=2)
plt.contour(xx, yy, np.transpose(zz3/zz), levels=0.9, colors='k',
           linestyles='-', linewidths=2)
plt.xlim(xlims)
plt.ylim(ylims)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('La$_t$')
plt.ylabel('$h_b/L_L$')
ax=plt.gca()
ax.set_aspect(aspect=1/3)
plt.text(0.11, 2e-3, 'Langmuir', bbox=dict(boxstyle="square",ec='k',fc='w'))
plt.text(3, 2e-3, 'Wind', bbox=dict(boxstyle="square",ec='k',fc='w'))
plt.text(0.13, 1e2, 'Convection', bbox=dict(boxstyle="square",ec='k',fc='w'))

# plot bi-dimensional histogram
plot_dist_4p(hist, xi, yi, colors='white', linestyles='-')

# reduce margin
plt.tight_layout()

# save figure
figname = fig_root+'/fig_regime_diagram_'+forcing_reg_type+'.png'
plt.savefig(figname, dpi = 300)

In [ ]:
# Figure 2: forcing regime

# plot figure
fig = plt.figure()
fig.set_size_inches(4.5, 4)

# range of power
xpr = [-1, 1]
ypr = [-3, 3]
# range
xlims = [10**i for i in xpr]
ylims = [10**i for i in ypr]
# background following Fig. 3 of Belcher et al., 2012
nx = 500
ny = 500
xx = np.logspace(xpr[0], xpr[1], nx)
yy = np.logspace(ypr[0], ypr[1], ny)
zz1 = np.zeros([nx, ny])
zz2 = np.zeros([nx, ny])
zz3 = np.zeros([nx, ny])
for i in np.arange(nx):
    for j in np.arange(ny):
        zz1[i,j] = 2*(1-np.exp(-0.5*xx[i]))
        zz2[i,j] = 0.22*xx[i]**(-2)
        zz3[i,j] = 0.3*xx[i]**(-2)*yy[j]
zz = zz1 + zz2 + zz3

rz_ST = zz1/zz
rz_LT = zz2/zz
rz_CT = zz3/zz
fr = np.ones(zz.shape) * 7
print(rz_LT.shape)
print(fr.shape)
cfrac = 0.25
fr[(rz_LT<cfrac) & (rz_CT<cfrac)] = 1
fr[(rz_ST<cfrac) & (rz_CT<cfrac)] = 2
fr[(rz_ST<cfrac) & (rz_LT<cfrac)] = 3
fr[(rz_ST>=cfrac) & (rz_LT>=cfrac) & (rz_CT<cfrac)] = 4
fr[(rz_ST>=cfrac) & (rz_CT>=cfrac) & (rz_LT<cfrac)] = 5
fr[(rz_LT>=cfrac) & (rz_CT>=cfrac) & (rz_ST<cfrac)] = 6

color_list = ['firebrick','forestgreen','royalblue','gold','orchid','turquoise','w']
cb_ticks = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5]
cmap, norm = from_levels_and_colors(cb_ticks, color_list)
plt.contourf(xx, yy, np.transpose(fr), cmap=cmap, norm=norm)
plt.contour(xx, yy, np.transpose(fr), colors='darkgray')
plt.xlim(xlims)
plt.ylim(ylims)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('La$_t$')
plt.ylabel('$h_b/L_L$')
ax=plt.gca()
ax.set_aspect(aspect=1/3)
plt.text(0.11, 2e-3, 'Langmuir', bbox=dict(boxstyle="square",ec='k',fc='w'))
plt.text(3, 2e-3, 'Wind', bbox=dict(boxstyle="square",ec='k',fc='w'))
plt.text(0.13, 1e2, 'Convection', bbox=dict(boxstyle="square",ec='k',fc='w'))

# plot bi-dimensional histogram
plot_dist_4p(hist, xi, yi, colors='white', linestyles='-')

# reduce margin
plt.tight_layout()

# save figure
figname = fig_root+'/fig_diag_forcing_regime_'+forcing_reg_type+'.png'
plt.savefig(figname, dpi = 300)